# 限值因变量模型

## 二值（或有限值）的结果

- 二值结果的例子到处可见（例如已婚妇女的劳动力市场参与、迁移决策等）

- 想要理解某个变量（无论是连续变量还是虚拟变量）对二值结果概率的影响

## 线性概率模型（Linear Probability Models）

模型

$$E(y|x)=P(y=1|x) \times 1 + P(y=0) \times 0= P(y=1|x) =\beta_{0}+\beta_{1} x_{1}+\beta_{2} x_{2}+\ldots+\beta_{k} x_{k}+\mu$$

其中$y$是二值变量（或虚拟变量），模型中解释变量前的系数则表示当$x$变化一个单位时，$y$成功概率的变动。

**例如**

我们构建下列模型

$$died_{i}=\beta_{0}+\beta_{1} age_{i}+\mu_{i}$$

请说明这个模型中系数的含义。

<br>

**讨论**：LPM模型存在的问题

- LPM模型并不是一个完全正确的模型。考虑对于线性模型而言，我们假定

$$Y \sim N\left(\beta_{0}+\beta_{1} X_{1}+\cdots+\beta_{p} X_{p}, \sigma^{2}\right)$$

这意味着以$X$为条件，$Y$服从正态分布。但毫无疑问，对于一个二值变量，它不可能是正态分布的。

- 异方差问题。当$y$是一个二值变量时，以$x$为条件的方差为 $\operatorname{var}(y | \mathbf{x})=p(\mathbf{x})[1-p(\mathbf{x})]$

- 如果代入自变量的某些特定组合数值，就能得到小于0或大于1的预测值

<br>

> 模型是有用的，特别是作为建模的起点

> 若结果的均值远离0或者1时，LPM和logit或probit模型没有大的差异

> 解释是简洁的

> 正态性假定只在推断时需要

**案例：孩子出生体重**

In [57]:
library(rio)
library(skimr)
library(tidyverse)
library(stargazer)

In [2]:
data <- import("http://www.stata-press.com/data/r14/lbw.dta")

data <- select(data, c(id, low, age, race, smoke, ht))
data$race <- factor(data$race)

In [3]:
skim(data)

-- Data Summary ------------------------
                           Values
Name                       data  
Number of rows             189   
Number of columns          6     
_______________________          
Column type frequency:           
  factor                   1     
  numeric                  5     
________________________         
Group variables            None  

-- Variable type: factor -------------------------------------------------------
# A tibble: 1 x 6
  skim_variable n_missing complete_rate ordered n_unique top_counts         
* <chr>             <int>         <dbl> <lgl>      <int> <chr>              
1 race                  0             1 FALSE          3 1: 96, 3: 67, 2: 26

-- Variable type: numeric ------------------------------------------------------
# A tibble: 5 x 11
  skim_variable n_missing complete_rate     mean     sd    p0   p25   p50   p75
* <chr>             <int>         <dbl>    <dbl>  <dbl> <dbl> <dbl> <dbl> <dbl>
1 id                    0  

In [4]:
lmobj <- lm(low ~ age + race + smoke + ht, data=data)
summary(lmobj)


Call:
lm(formula = low ~ age + race + smoke + ht, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.6173 -0.3369 -0.1355  0.4682  0.9158 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)   
(Intercept)  0.249336   0.169076   1.475  0.14201   
age         -0.005693   0.006333  -0.899  0.36987   
race2        0.184372   0.101615   1.814  0.07125 . 
race3        0.197529   0.077161   2.560  0.01128 * 
smoke        0.211903   0.071307   2.972  0.00336 **
ht           0.264236   0.133944   1.973  0.05003 . 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.4473 on 183 degrees of freedom
Multiple R-squared:  0.09792,	Adjusted R-squared:  0.07327 
F-statistic: 3.973 on 5 and 183 DF,  p-value: 0.001913


In [5]:
data$yhat <- lmobj$fitted.values
filter(data, yhat < 0)

id,low,age,race,smoke,ht,yhat
<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
226,0,45,1,0,0,-0.006855462


## 对数单位和概率单位模型（Logit or Probit model）

一个通常的考虑是寻找函数$F$，使得

$$p_{i}=P\left(y_{i}=1 | X_{1}, \ldots, X_{p}\right)=F\left(\beta_{0}+\beta_{1} X_{1}+\cdots+\beta_{p} X_{p}\right)$$

$F()$通常是累积分布函数，最常用的是logistic分布以及标准正态累积分布函数。若使用前者则得到logistic模型，后者则得到probit模型。

logistic响应函数为

$$\pi=\frac{e^{x}}{1+e^{x}}=\frac{1}{1+e^{-x}}$$

其中$x$是任意实数。

另一种考虑是利用潜变量模型。假设$y^{*}$是一个无法观测的变量或潜变量，有

$$y^{*}=\beta_{0}+x\beta+\mu, y=1\left[y^{*}>0\right]$$

其中$1[\cdot]$是一个指示函数，若$y^{*}>0$时$y$为1，否则为0。我们假设$\mu$是独立于$x$的，并且$\mu$符合logistic分布或者标准正态分布。这意味着$\mu$是以0为中心对称的，即对于任何实数$z$都有 $1-F(-z)=F(z)$。

因此有

$$\begin{aligned} \mathrm{P}(y=1 | \mathbf{x}) &=\mathrm{P}\left(y^{*}>0 | \mathbf{x}\right)=\mathrm{P}\left[\mu>-\left(\beta_{0}+\mathbf{x} \boldsymbol{\beta}\right) | \mathbf{x}\right] \\ &=1-F\left[-\left(\beta_{0}+\mathbf{x} \boldsymbol{\beta}\right)\right]=F\left(\beta_{0}+\mathbf{x} \boldsymbol{\beta}\right) \end{aligned}$$

### 估计方法 - 极大似然估计

鉴于Logit和Probit模型不再是线性模型，因此不能用OLS模型进行估计。常用的估计方法是极大似然估计（MLE）

#### 一个正态分布的案例

假设得到一组数，例如

90.46561
105.1319
117.5445
102.7179
102.7788
107.6234
94.87266
95.48918
75.63886
87.40594

我们认为来自于正态分布的总体，其中参数为$\mu$和$\sigma^{2}$，并且它们相互独立。我们需要从中猜测最可能的参数值。

我们知道正态分布的概率密度函数，考虑到这些观测值是独立的，所以联合分布函数可以写为

$$L\left(\mu, \sigma^{2}\right)=\prod_{i=1}^{n} \frac{1}{\sqrt{2 \pi \sigma^{2}}} \exp \left(\frac{-\left(y_{i}-\mu\right)^{2}}{2 \sigma^{2}}\right)$$

为了简化计算，写出它的对数形式

$$\log (L(x ; \mu, \sigma))=-\frac{1}{2} N \log (2 \pi)-N \log (\sigma)-\frac{1}{2} \sum_{i=1}^{N} \frac{\left(x_{i}-\mu\right)^{2}}{\sigma^{2}}$$

对未知参数求偏导等于$0$，即可解得极大似然估计量。

例如，对于$\mu$

$$\frac{\partial \ln \left(L\left(\mu, \sigma^{2}\right)\right)}{\partial \mu}=2 \frac{1}{2 \sigma^{2}} \sum_{i=1}^{n}\left(y_{i}-\mu\right)=0$$

解得$\hat{\mu}=\frac{\sum_{i=1}^{n} y_{i}}{n}=\bar{y}$，即$\mu$的极大似然估计量是样本均值。

In [41]:
library(maxLik)

set.seed(123)
x <- rnorm(100, mean = 1, sd = 2)

In [42]:
logLikFun <- function(param) {
    mu <- param[1]
    sigma <- param[2]
    sum(dnorm(x, mean = mu, sd = sigma, log = TRUE))
}

In [43]:
mle <- maxLik(logLik = logLikFun, start = c(mu = 0, sigma = 1))
summary(mle)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 7 iterations
Return code 1: gradient close to zero
Log-Likelihood: -201.5839 
2  free parameters
Estimates:
      Estimate Std. error t value  Pr(> t)    
mu      1.1808     0.1816   6.503 7.89e-11 ***
sigma   1.8165     0.1285  14.140  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
--------------------------------------------

或者也可以略改动代码

In [45]:
fn_mle <- function(x){
    force(x)
    function(param) {
        mu <- param[1]
        sigma <- param[2]
        sum((-1/2)*log(2*pi) - log(sigma) - (1/2)*((x-mu)^2/(sigma^2)))
    }
}

mle <- maxLik(logLik = fn_mle(x), start = c(mu = 0, sigma = 1))
summary(mle)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 7 iterations
Return code 1: gradient close to zero
Log-Likelihood: -201.5839 
2  free parameters
Estimates:
      Estimate Std. error t value  Pr(> t)    
mu      1.1808     0.1817    6.50 8.06e-11 ***
sigma   1.8165     0.1285   14.14  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
--------------------------------------------

**计量经济学导论的案例**

In [49]:
options(warn=-1)

# load wooldridge dataset
library("wooldridge")

data(wage1)

# regression
lmobj <- lm(lwage ~ educ + exper + tenure, data = wage1)
summary(lmobj)


Call:
lm(formula = lwage ~ educ + exper + tenure, data = wage1)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.05802 -0.29645 -0.03265  0.28788  1.42809 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.284360   0.104190   2.729  0.00656 ** 
educ        0.092029   0.007330  12.555  < 2e-16 ***
exper       0.004121   0.001723   2.391  0.01714 *  
tenure      0.022067   0.003094   7.133 3.29e-12 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.4409 on 522 degrees of freedom
Multiple R-squared:  0.316,	Adjusted R-squared:  0.3121 
F-statistic: 80.39 on 3 and 522 DF,  p-value: < 2.2e-16


In [56]:
ols.lf <- function(param) {
    beta <- param[-1] # Regression Coefficients
    sigma <- param[1] # Standard Deviation
    y <- as.vector(wage1$lwage) # DV
    x <- cbind(1, wage1$educ, wage1$exper, wage1$tenure) # IV
    mu <- x%*%beta # multiply matrices
    sum(dnorm(y, mu, sigma, log = TRUE)) # normal distribution(vector of observations, mean, sd)
}

mle_ols <- maxLik(logLik = ols.lf, start = c(sigma = 1, constant = 1, educ = 1, exper=1, tenure=1))
summary(mle_ols)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 16 iterations
Return code 1: gradient close to zero
Log-Likelihood: -313.5478 
5  free parameters
Estimates:
         Estimate Std. error t value  Pr(> t)    
sigma    0.439183   0.013541  32.434  < 2e-16 ***
constant 0.284360   0.103795   2.740  0.00615 ** 
educ     0.092029   0.007302  12.603  < 2e-16 ***
exper    0.004121   0.001717   2.401  0.01637 *  
tenure   0.022067   0.003082   7.160 8.05e-13 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
--------------------------------------------

**对数单位案例：已婚妇女的劳动力市场参与**

In [6]:
data <- import("https://p193.p3.n0.cdn.getcloudapp.com/items/kpu5Q9Br/MROZ.DTA")
skim(data)

-- Data Summary ------------------------
                           Values
Name                       data  
Number of rows             753   
Number of columns          22    
_______________________          
Column type frequency:           
  numeric                  22    
________________________         
Group variables            None  

-- Variable type: numeric ------------------------------------------------------
# A tibble: 22 x 11
   skim_variable n_missing complete_rate      mean         sd        p0
 * <chr>             <int>         <dbl>     <dbl>      <dbl>     <dbl>
 1 inlf                  0         1         0.568     0.496     0     
 2 hours                 0         1       741.      871.        0     
 3 kidslt6               0         1         0.238     0.524     0     
 4 kidsge6               0         1         1.35      1.32      0     
 5 age                   0         1        42.5       8.07     30     
 6 educ                  0         1        12.

我们想要估计下列logistic模型

$$\log \left(\frac{inlf_{i}}{1-inlf_{i}}\right)=\beta_{0}+\beta_{1} h s p_{i}$$

另外，我们也可以把模型写为

$$\log \left(\frac{inlf_{i}}{1-inlf_{i}}\right)=\beta_{0}+\beta_{1} h s p_{i}$$

或者

$$\operatorname{logit}\left(inlf_{i}\right)=\beta_{0}+\beta_{1} h s p_{i}$$

In [7]:
data$hsp = 0
data[data$educ > 12 & !is.na(data$educ), "hsp"] = 1
logit_model <- glm(inlf~hsp, family=binomial(link="logit"),data=data)
summary(logit_model)


Call:
glm(formula = inlf ~ hsp, family = binomial(link = "logit"), 
    data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5080  -1.2201   0.8795   1.1353   1.1353  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.09990    0.08609   1.160 0.245911    
hsp          0.65041    0.17048   3.815 0.000136 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1029.7  on 752  degrees of freedom
Residual deviance: 1014.7  on 751  degrees of freedom
AIC: 1018.7

Number of Fisher Scoring iterations: 4


- 对于$hsp = 1$的群体，其模型为$\log \left(\frac{inlf_{hsp}}{1-inlf_{hsp}}\right)=\beta_{0}+\beta_{1}$

- 对于$hsp = 0$的群体，其模型为$\log \left(\frac{inlf_{nohsp}}{1-inlf_{nohsp}}\right)=\beta_{0}$

两者的差异为

$$\log \left(\frac{inlf_{hsp}}{1-inlf_{hsp}}\right) - \log \left(\frac{inlf_{hsp}}{1-inlf_{hsp}}\right) = \beta_{1}$$

因此比值比为

$$\frac{\frac{inlf_{hsp}}{1-inlf_{hsp}}}{\frac{inlf_{nohsp}}{1-inlf_{nohsp}}}=e^{\beta_{1}}$$

在我们的例子里$e^{.6504074} = 1.92$。即大学毕业的群体进入劳动力市场的比率约为非大学群体的两倍。

考虑到对于$\log \left(\frac{p}{1-p}\right)=\beta_{0}+\beta_{1} x$，我们解得

$$p=\frac{e^{\beta_{0}+\beta_{1} x}}{1+e^{\beta_{0}+\beta_{1} x}}$$

因此，我们可以确切计算得到两个群体的概率。

In [8]:
ols_model <- lm(inlf~nwifeinc+educ+exper+expersq+age+kidslt6+kidsge6, data=data)
logit_model <- glm(inlf~nwifeinc+educ+exper+expersq+age+kidslt6+kidsge6, family=binomial(link="logit"),data=data)
probit_model <- glm(inlf~nwifeinc+educ+exper+expersq+age+kidslt6+kidsge6, family=binomial(link="probit"),data=data)

stargazer(ols_model, logit_model, probit_model, type="text", header = TRUE, digits = 4)


                                 Dependent variable:              
                    ----------------------------------------------
                                         inlf                     
                              OLS             logistic    probit  
                              (1)               (2)        (3)    
------------------------------------------------------------------
nwifeinc                   -0.0034**         -0.0213**  -0.0120** 
                            (0.0014)          (0.0084)   (0.0049) 
                                                                  
educ                       0.0380***         0.2212***  0.1309*** 
                            (0.0074)          (0.0434)   (0.0254) 
                                                                  
exper                      0.0395***         0.2059***  0.1233*** 
                            (0.0057)          (0.0321)   (0.0188) 
                                                             

In [9]:
library(margins)

summary(margins(logit_model))

,factor,AME,SE,z,p,lower,upper
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,age,-0.0157193592,0.0023807440,-6.6027087,4.037123e-11,-0.0203855317,-0.0110531867
2,educ,0.0394965219,0.0072946429,5.4144558,6.147537e-08,0.0251992846,0.0537937593
3,exper,0.0367640951,0.0051500235,7.1386267,9.426797e-13,0.0266702345,0.0468579557
4,expersq,-0.0005632587,0.0001773538,-3.1759039,1.493704e-03,-0.0009108657,-0.0002156516
5,kidsge6,0.0107348186,0.0133329508,0.8051345,4.207421e-01,-0.0153972847,0.0368669219
6,kidslt6,-0.2577536390,0.0319413851,-8.0695824,7.053902e-16,-0.3203576035,-0.1951496746
7,nwifeinc,-0.0038118134,0.0014823782,-2.5714177,1.012831e-02,-0.0067172212,-0.0009064056


In [10]:
summary(margins(probit_model))

,factor,AME,SE,z,p,lower,upper
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,age,-0.0158956635,0.0023586800,-6.7392199,1.592392e-11,-0.0205185915,-0.0112727356
2,educ,0.0393700931,0.0072657023,5.4186218,6.006021e-08,0.0251295782,0.0536106079
3,exper,0.0370973344,0.0051682526,7.1779260,7.077686e-13,0.0269677455,0.0472269233
4,expersq,-0.0005675459,0.0001770802,-3.2050227,1.350518e-03,-0.0009146167,-0.0002204752
5,kidsge6,0.0108288871,0.0132241237,0.8188737,4.128585e-01,-0.0150899191,0.0367476933
6,kidslt6,-0.2611534478,0.0319024151,-8.1860087,2.700322e-16,-0.3236810325,-0.1986258632
7,nwifeinc,-0.0036161755,0.0014697224,-2.4604479,1.387637e-02,-0.0064967785,-0.0007355724


对于连续变量，它对于因变量$p(\mathbf{x})=\mathrm{P}(y=1 | \mathbf{x})$的偏效应为

$$\frac{\partial p(\mathbf{x})}{\partial x_{j}}=f\left(\beta_{0}+\mathbf{x} \boldsymbol{\beta}\right) \beta_{j}$$

其中$f(z) \equiv \frac{d F}{d z}(z)$

一般而言，有两种衡量偏效应

- 在均值的偏效应（PEA）

$$f\left(\hat{\beta}_{0}+\overline{\mathbf{x}} \hat{\boldsymbol{\beta}}\right)=f\left(\hat{\beta}_{0}+\hat{\beta}_{1} \overline{x}_{1}+\hat{\beta}_{2} \overline{x}_{2}+\ldots+\hat{\beta}_{k} \overline{x}_{k}\right)$$

- 平均偏效应（APE）

$$n^{-1} \sum_{i=1}^{n}\left[f\left(\hat{\beta}_{0}+\mathbf{x}_{i} \hat{\boldsymbol{\beta}}\right) \hat{\beta}_{j}\right]=\left[n^{-1} \sum_{i=1}^{n} f\left(\hat{\beta}_{0}+\mathbf{x}_{i} \hat{\boldsymbol{\beta}}\right)\right] \hat{\beta}_{j}$$

为了简便计算，也可以使用粗略计算。对于概率单位而言，$f(0)\approx0.4$，对于对数单位而言，$f(0)\approx0.25$。在上述例子中，这些比例因子约为$0.301$（概率单位）和$0.179$（对数单位）。

### 模型评价

我们无法像线性回归模型一样把方差分解为解释和不能解释的部分，因此惯常的$R^{2}$无法使用。

#### 伪$R^{2}$

- 使用似然函数进行比较

- 比较当前模型与不包括任何解释变量的模型

- 伪$R^{2}$的公式为$1-\frac{ll_{cm}}{ll_{null}}$，其中$ll_{cm}$是当前模型的对数似然函数，$ll_{null}$是零模型的对数似然函数

- 如果当前模型与零模型一样好，那么$\frac{ll_{cm}}{ll_{null}}$趋于$1$，则伪$R^{2}$就趋于$0$；否则，它就大于$0$

- 伪$R^{2}$并不是衡量模型对于数据的拟合有多好，而是和零模型相比有多好

#### 正确预测百分比

一个拟合优度的指标

In [35]:
# predict value of logit and probit model
logit_y_hat <- predict(logit_model, data, type="response")
probit_y_hat <- predict(probit_model, data, type="response")

In [39]:
# to calculate percent correctly predicted
logit_y_predict <- rep(1, length(logit_y_hat))
logit_y_predict[logit_y_hat < 0.5] <- 0
logit_pcp <- sum(logit_y_predict == data$inlf) / length(logit_y_predict) * 100

probit_y_predict <- rep(1, length(probit_y_hat))
probit_y_predict[probit_y_hat < 0.5] <- 0
probit_pcp <- sum(probit_y_predict == data$inlf) / length(probit_y_predict) * 100

sprintf("Percent correctly predicted of logit model is %.2f %%", logit_pcp)
sprintf("Percent correctly predicted of probit model is %.2f %%", probit_pcp)

[1] "Percent correctly predicted of logit model is 73.57 %"

[1] "Percent correctly predicted of probit model is 73.44 %"